<a href="https://colab.research.google.com/github/aruaru0/Foursquare-Location-Matching/blob/main/train_lightgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/datas/kaggle.json  ~/.kaggle/

In [ ]:
!pip install kaggle
!apt install unzip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!mkdir input output

In [ ]:
!kaggle competitions download -c foursquare-location-matching

100% 158M/159M [00:01<00:00, 167MB/s]
100% 159M/159M [00:01<00:00, 161MB/s]


In [ ]:
!unzip foursquare-location-matching.zip -d input/foursquare-location-matching

Archive:  foursquare-location-matching.zip
  inflating: input/foursquare-location-matching/pairs.csv  
  inflating: input/foursquare-location-matching/sample_submission.csv  
  inflating: input/foursquare-location-matching/test.csv  
  inflating: input/foursquare-location-matching/train.csv  


In [ ]:
!kaggle kernels output aruaru0/training-data-for-binary-lgb-baseline-0-834 -p ./input/train_data

tcmalloc: large alloc 1212923904 bytes == 0x59a84000 @  0x7f2fa17c01e7 0x4a3940 0x5b438c 0x5b46f7 0x59afff 0x515655 0x59a257 0x570bf0 0x511ee1 0x549576 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576 0x4bca8a 0x5134a6 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x511e2c 0x549576 0x4bca8a
Output file downloaded to ./input/train_data/train_data1.csv
tcmalloc: large alloc 1215537152 bytes == 0xa28c2000 @  0x7f2fa17c01e7 0x4a3940 0x5b438c 0x5b46f7 0x59afff 0x515655 0x59a257 0x570bf0 0x511ee1 0x549576 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576 0x4bca8a 0x5134a6 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x511e2c 0x549576 0x4bca8a
Output file downloaded to ./input/train_data/train_data2.csv
Output file downloaded to ./input/train_data/train_data3.csv
Output file downloaded to ./input/train_data/train_data4.csv
Output file downloaded to ./input/train_data/train_data5.csv
Output file d

In [ ]:
# !pip uninstall --yes lightgbm && pip install --install-option=--gpu lightgbm

## lightgbm

In [ ]:
import lightgbm as lgb

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_sample_weight
import seaborn as sns
import pickle
import gc

In [ ]:
class CFG:
    seed = 46
    target = "point_of_interest"
    n_splits = 3

    n_neighbors = 25
    is_debug = False

    model_path = '/content/drive/MyDrive/datas/foursquare'

In [ ]:
def reduce_memory(df):
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            cmin = df[col].min()
            cmax = df[col].max()
            if str(col_type)[:3] == 'int':
                if cmin > np.iinfo(np.int8).min and cmax < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif cmin > np.iinfo(np.int16).min and cmax < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif cmin > np.iinfo(np.int32).min and cmax < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif cmin > np.iinfo(np.int64).min and cmax < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if cmin > np.finfo(np.float16).min and cmax < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif cmin > np.finfo(np.float32).min and cmax < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    return df

In [ ]:
df1 = reduce_memory(pd.read_csv('/content/input/train_data/train_data1.csv'))
df2 = reduce_memory(pd.read_csv('/content/input/train_data/train_data2.csv'))
df3 = reduce_memory(pd.read_csv('/content/input/train_data/train_data3.csv'))
df4 = reduce_memory(pd.read_csv('/content/input/train_data/train_data4.csv'))
df5 = reduce_memory(pd.read_csv('/content/input/train_data/train_data5.csv'))

In [ ]:
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
from tqdm import tqdm

def add_fold(df) :
  kf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
  for i, (trn_idx, val_idx) in tqdm(enumerate(kf.split(df, df["label"], df["label"]))):
      df.loc[val_idx, "fold"] = i
  return df


In [ ]:
df1 = add_fold(df1)
df2 = add_fold(df2)
df3 = add_fold(df3)
df4 = add_fold(df4)
df5 = add_fold(df5)

3it [00:00,  5.11it/s]
3it [00:00,  5.13it/s]
3it [00:00,  5.20it/s]
3it [00:00,  5.20it/s]
3it [00:00,  5.25it/s]


In [ ]:
train = pd.concat([df1,df2,df3,df4,df5])
train = train.reset_index(drop=True)

In [ ]:
train.head()

,id,match_id,kdist,kneighbors,kdist_country,kneighbors_country,label,name_sim,name_gesh,name_leven,...,country_sim,country_gesh,country_leven,country_jaro,country_lcs,country_len_diff,country_nleven,country_nlcsk,country_nlcs,fold
0,E_000001272c6c5d,E_000001272c6c5d,0.0,0.0,0.0,0.0,1,1.0,1.0,0.0,...,1.0,1.0,0.0,1.0,2.0,0,0.0,1.0,1.0,1.0
1,E_00009ab517afac,E_00009ab517afac,0.0,0.0,0.0,0.0,1,1.0,1.0,0.0,...,1.0,1.0,0.0,1.0,2.0,0,0.0,1.0,1.0,1.0
2,E_0000c362229d93,E_0000c362229d93,0.0,0.0,0.0,0.0,1,1.0,1.0,0.0,...,1.0,1.0,0.0,1.0,2.0,0,0.0,1.0,1.0,1.0
3,E_0000c566a81ea1,E_0000c566a81ea1,0.0,0.0,0.0,0.0,1,1.0,1.0,0.0,...,1.0,1.0,0.0,1.0,2.0,0,0.0,1.0,1.0,1.0
4,E_0000d9e584ed9f,E_0000d9e584ed9f,0.0,0.0,0.0,0.0,1,1.0,1.0,0.0,...,1.0,1.0,0.0,1.0,2.0,0,0.0,1.0,1.0,1.0


In [ ]:
del df1, df2, df3, df4, df5
gc.collect()

50

In [ ]:
def fit_lgbm(X, y, params=None, es_rounds=20, seed=42, N_SPLITS=5, 
             n_class=None, model_dir=None, folds=None):
  models = []
  oof = np.zeros((len(y), n_class), dtype=np.float64)
  
  for i in tqdm(range(CFG.n_splits)):
    print(f"== fold {i} ==")
    trn_idx = folds!=i
    val_idx = folds==i
    # train, validデータに分割
    X_train, y_train = X[trn_idx], y.iloc[trn_idx]
    X_valid, y_valid = X.iloc[val_idx], y.iloc[val_idx]

    if model_dir is None:
      # 学習
      model = lgb.LGBMClassifier(**params)
      model.fit(
          X_train, y_train, 
          eval_set=[(X_valid, y_valid)],  
          early_stopping_rounds=es_rounds, 
          eval_metric='logloss',  
#             verbose=-1)
          verbose=50)
    else: 
      # 既に学習している場合は、読み込む（トレーニングをしない）
      with open(f'{model_dir}/lgbm_fold{i}.pkl', 'rb') as f:
          model = pickle.load(f)

    # validデータに対して予測し、結果を格納    
    pred = model.predict_proba(X_valid)
    oof[val_idx] = pred
    models.append(model)
    
    # 学習した結果をファイルに保存（！！！colabの場合は、、google driveに保存するように変更しておくしておく！！！）
    #file = f'lgbm_fold{i}.pkl'
    file = f'{CFG.model_path}/lgbm_fold{i}.pkl'

    pickle.dump(model, open(file, 'wb'))
    print()
    model.booster_.save_model(f'{CFG.model_path}/lgbm_fold{i}.txt')

  cv = (oof.argmax(axis=-1) == y).mean()
  print(f"CV-accuracy: {cv}")

  return oof, models


In [ ]:
features = ['kdist', 'kneighbors', 'kdist_country',
        'kneighbors_country', 'name_sim', 'name_gesh', 'name_leven',
        'name_jaro', 'name_lcs', 'name_len_diff', 'name_nleven', 'name_nlcsk',
        'name_nlcs', 'address_sim', 'address_gesh', 'address_leven',
        'address_jaro', 'address_lcs', 'address_len_diff', 'address_nleven',
        'address_nlcsk', 'address_nlcs', 'city_gesh', 'city_leven', 'city_jaro',
        'city_lcs', 'city_len_diff', 'city_nleven', 'city_nlcsk', 'city_nlcs',
        'state_sim', 'state_gesh', 'state_leven', 'state_jaro', 'state_lcs',
        'state_len_diff', 'state_nleven', 'state_nlcsk', 'state_nlcs',
        'zip_gesh', 'zip_leven', 'zip_jaro', 'zip_lcs', 'url_sim', 'url_gesh',
        'url_leven', 'url_jaro', 'url_lcs', 'url_len_diff', 'url_nleven',
        'url_nlcsk', 'url_nlcs', 'phone_gesh', 'phone_leven', 'phone_jaro',
        'phone_lcs', 'categories_sim', 'categories_gesh', 'categories_leven',
        'categories_jaro', 'categories_lcs', 'categories_len_diff',
        'categories_nleven', 'categories_nlcsk', 'categories_nlcs',
        'country_sim', 'country_gesh', 'country_leven', 'country_jaro',
        'country_lcs', 'country_len_diff', 'country_nleven', 'country_nlcsk',
        'country_nlcs']

In [ ]:
params = {
    'objective': "binary",
    'learning_rate': 0.2,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'random_state': 42,

    'max_depth': 7,   
    'num_leaves': 35, 
    'n_estimators': 1000000, 
    "colsample_bytree": 0.9,
}

oof, models = fit_lgbm(train[features], train["label"].astype(int), 
                      params=params, 
                      n_class=int(train["label"].max() + 1), 
                      N_SPLITS=CFG.n_splits, 
                      folds=train["fold"].values)

  0%|          | 0/3 [00:00<?, ?it/s]

== fold 0 ==
Training until validation scores don't improve for 20 rounds.
[50]	valid_0's binary_logloss: 0.0437441	valid_0's binary_logloss: 0.0437441
[100]	valid_0's binary_logloss: 0.0419036	valid_0's binary_logloss: 0.0419036
[150]	valid_0's binary_logloss: 0.0410559	valid_0's binary_logloss: 0.0410559
[200]	valid_0's binary_logloss: 0.0403682	valid_0's binary_logloss: 0.0403682
[250]	valid_0's binary_logloss: 0.0399543	valid_0's binary_logloss: 0.0399543
[300]	valid_0's binary_logloss: 0.0395502	valid_0's binary_logloss: 0.0395502
[350]	valid_0's binary_logloss: 0.0392057	valid_0's binary_logloss: 0.0392057
[400]	valid_0's binary_logloss: 0.0389056	valid_0's binary_logloss: 0.0389056
[450]	valid_0's binary_logloss: 0.0386122	valid_0's binary_logloss: 0.0386122
[500]	valid_0's binary_logloss: 0.0383548	valid_0's binary_logloss: 0.0383548
[550]	valid_0's binary_logloss: 0.0381267	valid_0's binary_logloss: 0.0381267
[600]	valid_0's binary_logloss: 0.0378883	valid_0's binary_logloss: 

 33%|███▎      | 1/3 [47:43<1:35:26, 2863.17s/it]

== fold 1 ==
Training until validation scores don't improve for 20 rounds.
[50]	valid_0's binary_logloss: 0.0433894	valid_0's binary_logloss: 0.0433894
[100]	valid_0's binary_logloss: 0.0415704	valid_0's binary_logloss: 0.0415704
[150]	valid_0's binary_logloss: 0.0407402	valid_0's binary_logloss: 0.0407402
[200]	valid_0's binary_logloss: 0.0401077	valid_0's binary_logloss: 0.0401077
[250]	valid_0's binary_logloss: 0.0396587	valid_0's binary_logloss: 0.0396587
[300]	valid_0's binary_logloss: 0.0392306	valid_0's binary_logloss: 0.0392306
[350]	valid_0's binary_logloss: 0.0388946	valid_0's binary_logloss: 0.0388946
[400]	valid_0's binary_logloss: 0.0386564	valid_0's binary_logloss: 0.0386564
[450]	valid_0's binary_logloss: 0.0383892	valid_0's binary_logloss: 0.0383892
[500]	valid_0's binary_logloss: 0.0381378	valid_0's binary_logloss: 0.0381378
[550]	valid_0's binary_logloss: 0.0378487	valid_0's binary_logloss: 0.0378487
[600]	valid_0's binary_logloss: 0.0376354	valid_0's binary_logloss: 

 67%|██████▋   | 2/3 [1:36:17<48:13, 2893.11s/it]

== fold 2 ==
Training until validation scores don't improve for 20 rounds.
[50]	valid_0's binary_logloss: 0.0432583	valid_0's binary_logloss: 0.0432583
[100]	valid_0's binary_logloss: 0.0414234	valid_0's binary_logloss: 0.0414234
[150]	valid_0's binary_logloss: 0.0404689	valid_0's binary_logloss: 0.0404689
[200]	valid_0's binary_logloss: 0.0398697	valid_0's binary_logloss: 0.0398697
[250]	valid_0's binary_logloss: 0.0393719	valid_0's binary_logloss: 0.0393719
[300]	valid_0's binary_logloss: 0.0389979	valid_0's binary_logloss: 0.0389979
[350]	valid_0's binary_logloss: 0.0387224	valid_0's binary_logloss: 0.0387224
[400]	valid_0's binary_logloss: 0.0383863	valid_0's binary_logloss: 0.0383863
[450]	valid_0's binary_logloss: 0.0381021	valid_0's binary_logloss: 0.0381021
[500]	valid_0's binary_logloss: 0.0378512	valid_0's binary_logloss: 0.0378512
[550]	valid_0's binary_logloss: 0.0376156	valid_0's binary_logloss: 0.0376156
[600]	valid_0's binary_logloss: 0.0373844	valid_0's binary_logloss: 

100%|██████████| 3/3 [2:36:28<00:00, 3129.34s/it]

CV-accuracy: 0.9912986427324357


In [ ]:
# import gc
# del df1, df2, df3, df4, df5 
# gc.collect()

In [ ]:
# def set_datas(train, valid, test) :
#   df = pd.concat(train)
#   x_train = df[['kdist', 'kneighbors', 'kdist_country',
#         'kneighbors_country', 'name_sim', 'name_gesh', 'name_leven',
#         'name_jaro', 'name_lcs', 'name_len_diff', 'name_nleven', 'name_nlcsk',
#         'name_nlcs', 'address_sim', 'address_gesh', 'address_leven',
#         'address_jaro', 'address_lcs', 'address_len_diff', 'address_nleven',
#         'address_nlcsk', 'address_nlcs', 'city_gesh', 'city_leven', 'city_jaro',
#         'city_lcs', 'city_len_diff', 'city_nleven', 'city_nlcsk', 'city_nlcs',
#         'state_sim', 'state_gesh', 'state_leven', 'state_jaro', 'state_lcs',
#         'state_len_diff', 'state_nleven', 'state_nlcsk', 'state_nlcs',
#         'zip_gesh', 'zip_leven', 'zip_jaro', 'zip_lcs', 'url_sim', 'url_gesh',
#         'url_leven', 'url_jaro', 'url_lcs', 'url_len_diff', 'url_nleven',
#         'url_nlcsk', 'url_nlcs', 'phone_gesh', 'phone_leven', 'phone_jaro',
#         'phone_lcs', 'categories_sim', 'categories_gesh', 'categories_leven',
#         'categories_jaro', 'categories_lcs', 'categories_len_diff',
#         'categories_nleven', 'categories_nlcsk', 'categories_nlcs',
#         'country_sim', 'country_gesh', 'country_leven', 'country_jaro',
#         'country_lcs', 'country_len_diff', 'country_nleven', 'country_nlcsk',
#         'country_nlcs']]
#   y_train = df['label']

#   df = pd.concat(valid)
#   x_valid = df[['kdist', 'kneighbors', 'kdist_country',
#         'kneighbors_country', 'name_sim', 'name_gesh', 'name_leven',
#         'name_jaro', 'name_lcs', 'name_len_diff', 'name_nleven', 'name_nlcsk',
#         'name_nlcs', 'address_sim', 'address_gesh', 'address_leven',
#         'address_jaro', 'address_lcs', 'address_len_diff', 'address_nleven',
#         'address_nlcsk', 'address_nlcs', 'city_gesh', 'city_leven', 'city_jaro',
#         'city_lcs', 'city_len_diff', 'city_nleven', 'city_nlcsk', 'city_nlcs',
#         'state_sim', 'state_gesh', 'state_leven', 'state_jaro', 'state_lcs',
#         'state_len_diff', 'state_nleven', 'state_nlcsk', 'state_nlcs',
#         'zip_gesh', 'zip_leven', 'zip_jaro', 'zip_lcs', 'url_sim', 'url_gesh',
#         'url_leven', 'url_jaro', 'url_lcs', 'url_len_diff', 'url_nleven',
#         'url_nlcsk', 'url_nlcs', 'phone_gesh', 'phone_leven', 'phone_jaro',
#         'phone_lcs', 'categories_sim', 'categories_gesh', 'categories_leven',
#         'categories_jaro', 'categories_lcs', 'categories_len_diff',
#         'categories_nleven', 'categories_nlcsk', 'categories_nlcs',
#         'country_sim', 'country_gesh', 'country_leven', 'country_jaro',
#         'country_lcs', 'country_len_diff', 'country_nleven', 'country_nlcsk',
#         'country_nlcs']]
#   y_valid = df['label']

#   df = pd.concat(test)
#   x_test = df[['kdist', 'kneighbors', 'kdist_country',
#         'kneighbors_country', 'name_sim', 'name_gesh', 'name_leven',
#         'name_jaro', 'name_lcs', 'name_len_diff', 'name_nleven', 'name_nlcsk',
#         'name_nlcs', 'address_sim', 'address_gesh', 'address_leven',
#         'address_jaro', 'address_lcs', 'address_len_diff', 'address_nleven',
#         'address_nlcsk', 'address_nlcs', 'city_gesh', 'city_leven', 'city_jaro',
#         'city_lcs', 'city_len_diff', 'city_nleven', 'city_nlcsk', 'city_nlcs',
#         'state_sim', 'state_gesh', 'state_leven', 'state_jaro', 'state_lcs',
#         'state_len_diff', 'state_nleven', 'state_nlcsk', 'state_nlcs',
#         'zip_gesh', 'zip_leven', 'zip_jaro', 'zip_lcs', 'url_sim', 'url_gesh',
#         'url_leven', 'url_jaro', 'url_lcs', 'url_len_diff', 'url_nleven',
#         'url_nlcsk', 'url_nlcs', 'phone_gesh', 'phone_leven', 'phone_jaro',
#         'phone_lcs', 'categories_sim', 'categories_gesh', 'categories_leven',
#         'categories_jaro', 'categories_lcs', 'categories_len_diff',
#         'categories_nleven', 'categories_nlcsk', 'categories_nlcs',
#         'country_sim', 'country_gesh', 'country_leven', 'country_jaro',
#         'country_lcs', 'country_len_diff', 'country_nleven', 'country_nlcsk',
#         'country_nlcs']]
#   y_test = df['label']
#   del df
#   return x_train, y_train, x_valid, y_valid, x_test, y_test

In [ ]:
# def train(train, valid, test, outfile): 
#   x_train, y_train, x_valid, y_valid, x_test, y_test = set_datas(train, valid, test)

#   train_data = lgb.Dataset(
#       data=x_train, 
#       label=y_train, 
#       weight=compute_sample_weight(class_weight='balanced', y=y_train).astype('float32'),
#   )
#   validation_data = lgb.Dataset(
#       data=x_valid, 
#       label=y_valid, 
#       weight=np.ones(len(x_valid)).astype('float32'),
#   )
#   params = {
#     'task': 'train',
#     'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'metric': 'binary_logloss',
#     'num_leaves': 64,
#     'min_data_in_leaf': 20,
#     'max_depth': 7,
#     'verbose': 0,

#     'learning_rate': 0.2,
#     'random_state': 42,
#   }
#   # params = {
#   #   'objective': "logloss",
#   #   'learning_rate': 0.2,
#   #   'reg_alpha': 0.1,
#   #   'reg_lambda': 0.1,
#   #   'random_state': 42,

#   #   'max_depth': 7,   
#   #   'num_leaves': 35, 
#   #   'n_estimators': 1000000, 
#   #   "colsample_bytree": 0.9,
#   # }

#   bst = lgb.train(
#     params=params, 
#     train_set=train_data, 
#     num_boost_round=100, 
#     early_stopping_rounds=20,
#     valid_sets=[validation_data],
#   )
#   bst.save_model(outfile, num_iteration=bst.best_iteration)

#   y_pred = bst.predict(x_test, num_iteration=bst.best_iteration)
#   y_pred = y_pred.round(0)
#   accuracy = accuracy_score(y_pred, y_test)
#   print(f"accuracy score: {accuracy:0.4f}")
#   # accuracy score: 0.9825
#   cm = confusion_matrix(y_test, y_pred)
#   cm_matrix = pd.DataFrame(data=cm, columns=['Actual Positive:1', 'Actual Negative:0'], 
#                                   index=['Predict Positive:1', 'Predict Negative:0'])

#   sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')
#   del x_train, y_train, x_valid, y_valid, x_test, y_test, bst, y_pred, cm_matrix

In [ ]:
# train([df1,df2,df3,df4], [df5], [df5], '/content/drive/MyDrive/datas/foursquare/model1.txt')
# gc.collect()

In [ ]:
# train([df2,df3,df4], [df5], [df1], '/content/drive/MyDrive/datas/foursquare/model2.txt')
# gc.collect()

In [ ]:
# train([df3,df4,df5], [df1], [df2], '/content/drive/MyDrive/datas/foursquare/model3.txt')
# gc.collect()

In [ ]:
# train([df4,df5,df1], [df2], [df3], '/content/drive/MyDrive/datas/foursquare/model4.txt')
# gc.collect()

In [ ]:
# train([df5,df1,df2], [df3], [df4], '/content/drive/MyDrive/datas/foursquare/model5.txt')
# gc.collect()

In [ ]:
# df = df1
# x_test = df[['kdist', 'kneighbors', 'kdist_country',
#       'kneighbors_country', 'name_sim', 'name_gesh', 'name_leven',
#       'name_jaro', 'name_lcs', 'name_len_diff', 'name_nleven', 'name_nlcsk',
#       'name_nlcs', 'address_sim', 'address_gesh', 'address_leven',
#       'address_jaro', 'address_lcs', 'address_len_diff', 'address_nleven',
#       'address_nlcsk', 'address_nlcs', 'city_gesh', 'city_leven', 'city_jaro',
#       'city_lcs', 'city_len_diff', 'city_nleven', 'city_nlcsk', 'city_nlcs',
#       'state_sim', 'state_gesh', 'state_leven', 'state_jaro', 'state_lcs',
#       'state_len_diff', 'state_nleven', 'state_nlcsk', 'state_nlcs',
#       'zip_gesh', 'zip_leven', 'zip_jaro', 'zip_lcs', 'url_sim', 'url_gesh',
#       'url_leven', 'url_jaro', 'url_lcs', 'url_len_diff', 'url_nleven',
#       'url_nlcsk', 'url_nlcs', 'phone_gesh', 'phone_leven', 'phone_jaro',
#       'phone_lcs', 'categories_sim', 'categories_gesh', 'categories_leven',
#       'categories_jaro', 'categories_lcs', 'categories_len_diff',
#       'categories_nleven', 'categories_nlcsk', 'categories_nlcs',
#       'country_sim', 'country_gesh', 'country_leven', 'country_jaro',
#       'country_lcs', 'country_len_diff', 'country_nleven', 'country_nlcsk',
#       'country_nlcs']]
# y_test = df['label']

In [ ]:
# bst = lgb.Booster(model_file = '/content/drive/MyDrive/datas/foursquare/improved_lgb_baseline.lgb')

# y_pred = bst.predict(x_test, num_iteration=bst.best_iteration)
# y_pred = y_pred.round(0)
# accuracy = accuracy_score(y_pred, y_test)
# print(f"accuracy score: {accuracy:0.4f}")
# # accuracy score: 0.9825
# cm = confusion_matrix(y_test, y_pred)
# cm_matrix = pd.DataFrame(data=cm, columns=['Actual Positive:1', 'Actual Negative:0'], 
#                                 index=['Predict Positive:1', 'Predict Negative:0'])

# sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')